In [25]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import operator
import os
from PIL import Image
from tqdm import tqdm_notebook as tqdm

from shutil import copyfile
import pickle

# keras : librairie de deep learning
import keras
from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, InputLayer, ReLU, AveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.callbacks import ReduceLROnPlateau

# Lecture Images

In [13]:
def sep_train_test(new_paths = ['Split_Data/Airliners/Train', 'Split_Data/Airliners/Test'], path = 'G7_scrapping/Airliners/data', y = ['Int', 'Ext_Int', 'Ext', 'Meal'], split_limite = .7, limite = 400):
    os.makedirs(new_paths[0], exist_ok = True)
    os.makedirs(new_paths[1], exist_ok = True)
    
    for fd in os.listdir(new_paths[0]):
        shutil.rmtree(new_paths[0] + '/' + fd, ignore_errors=True)
        
    for fd in os.listdir(new_paths[1]):
        shutil.rmtree(new_paths[1] + '/' + fd, ignore_errors=True)
    
    for lab in y:
        os.makedirs(new_paths[0] + '/' + lab)
        os.makedirs(new_paths[1] + '/' + lab)
        
        counter = 0
        for pict in os.listdir(path + '/' + lab):
            counter += 1
            if counter <= limite:
                rand = np.random.random()
                if rand <= split_limite:
                    copyfile(path + '/' + lab + '/' + pict, new_paths[0] + '/' + lab + '/' + pict)
                else:
                    copyfile(path + '/' + lab + '/' + pict, new_paths[1] + '/' + lab + '/' + pict)

In [14]:
sep_train_test(new_paths = ['Split_Data/SeatGuru/Train', 'Split_Data/SeatGuru/Test'], path = 'IMG_annot/SEATGURU')

In [15]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

In [16]:
train_generator = train_datagen.flow_from_directory('Split_Data/SeatGuru/Train',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 713 images belonging to 4 classes.


In [17]:
train_generator.class_indices

{'Ext': 0, 'Ext_Int': 1, 'Int': 2, 'Meal': 3}

In [18]:
def save_obj(obj, name ):
    with open('Models/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('Models/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [19]:
save_obj(train_generator.class_indices, 'dict_model_view_13_01')

In [20]:
test_generator = train_datagen.flow_from_directory('Split_Data/SeatGuru/Test',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 311 images belonging to 4 classes.


# Transfert Learning

In [23]:
# create the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# add a global spatial average pooling layer
x = base_model.output



x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Conv2D(256, kernel_size=(3, 3))(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)

# let's add a fully-connected layer
x = Dense(256)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

x = Dense(128)(x)
x = keras.layers.BatchNormalization()(x)
x = ReLU()(x)

# dernière couche que sert a prédire la bonne classe
predictions = Dense(4, activation = 'softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [26]:
step_size_train=train_generator.n//train_generator.batch_size

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              patience=1, 
                              verbose=1, 
                              factor=0.5, 
                              min_lr=0.00001)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=5,
                   validation_data = test_generator,
                   callbacks=[reduce_lr])

Epoch 1/5
22/22 [==============================] - 216s 10s/step - loss: 0.4103 - accuracy: 0.8576 - val_loss: 0.0426 - val_accuracy: 0.8264
Epoch 2/5
22/22 [==============================] - 220s 10s/step - loss: 0.0886 - accuracy: 0.9838 - val_loss: 0.2776 - val_accuracy: 0.8875
Epoch 3/5
22/22 [==============================] - 236s 11s/step - loss: 0.0412 - accuracy: 0.9915 - val_loss: 0.0792 - val_accuracy: 0.9421
Epoch 4/5
22/22 [==============================] - 233s 11s/step - loss: 0.0250 - accuracy: 0.9924 - val_loss: 0.3139 - val_accuracy: 0.9453
Epoch 5/5
22/22 [==============================] - 229s 10s/step - loss: 0.0411 - accuracy: 0.9941 - val_loss: 0.2916 - val_accuracy: 0.9164

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.


In [27]:
model.save('model_view_14_01.h5')